# Scraping the bottom of the barrel

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import time
import urllib
import re
import os

from providers import ParariusProvider
from utils import Entry
import webbrowser

import pickle

In [ ]:
urls = {
  'Pararius Den Haag':  "/apartments/den-haag/0-1750/2-bedrooms/since-3",
  'Pararius Leidschendam': "/apartments/leidschendam/0-1750/2-bedrooms/since-3",
  'Pararius Rijswijk': "/apartments/rijswijk/0-1750/2-bedrooms/since-3",
  'Pararius Leiden': "/apartments/leiden/0-1750/2-bedrooms/since-3",
  'Pararius Wassenaar': "/apartments/Wassenaar/0-1750/2-bedrooms/since-3",
 # 'Pararius Utrecht': "/apartments/utrecht/0-1750/2-bedrooms",
  'Pararius Leidedorp': "/apartments/leiderdorp/0-1750/2-bedrooms/since-3",
  'Pararius Oegstgeest': "/apartments/oegstgeest/0-1750/2-bedrooms/since-3",
  'Pararius Rijnsburg': "/apartments/rijnsburg/0-1750/2-bedrooms/since-3",
 # 'Pararius Delft': "/apartments/delft/0-1750/2-bedrooms",
  'Pararius Voorburg': "/apartments/voorburg/0-1750/2-bedrooms/since-3"
}

# Too good to be true threshold (probably fake listing)
tgtbt_area_price_threshold = 13
# Tiny Room Threshold (the rooms are just too small)
tiny_room_threshold = 15

In [ ]:
pps = [ParariusProvider(urls[url], url.removeprefix('Pararius ')) for url in urls]
       
for pp in pps:
  pp.query_entries()
  pp.get_detailed_results()

In [ ]:
entries = []
for pp in pps:
  entries += pp.entries

len(entries)

In [ ]:
import glob
len(glob.glob1('.',"*.html"))

In [ ]:
df = pd.DataFrame(entries)

df['Price per area'] = df['price'] / df['area']
df['Price per room'] = df['price'] / df['rooms']
df['Area per room'] = df['area'] / df['rooms']

initial_rows = df.shape[0]

df.head()

In [ ]:
#df = df[df['Price per area'] > tgtbt_area_price_threshold]
df = df[df['Area per room'] > tiny_room_threshold]
df = df[~df["description"].str.contains('permit', case=False)]
df = df[df["rented"].isnull()]

print(f"{df.shape[0]}/{initial_rows} rows eligible")

In [ ]:
output_file_name = "output.dat"

array = []

if os.path.isfile(output_file_name):
  with open(output_file_name, "r") as f:
    array = f.read()[1:-1].replace("'","").strip().split(', ')
    df2 = df[~df["link"].isin(array)]
    print(f"{df2.shape[0]} new Properties!")
  
with open(output_file_name, "w") as f:
  array += list(df2["link"])
  f.write(str(array))

df2.head()

In [ ]:
links = list(df2.link)
for link in links:
  webbrowser.open_new_tab(link)

In [ ]:
import smtplib, ssl

from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage

In [ ]:
def prep_description(description):
  description = description.removeprefix('Description\n')

def truncate(msg):
  length = 125
  if len(msg) < length:
    return msg
  new_msg = ''
  for word in msg.split(' '):
    if len(new_msg + word) > length:
      new_msg += '...'
      break

    new_msg += word + ' '

  return new_msg

In [ ]:
list(df3['description'].apply(truncate))

In [ ]:
if links:
  port = 465  # For SSL
  password = 'xrko gsmg xivc oshm'

  # Create a secure SSL context
  context = ssl.create_default_context()

  with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
    server.login("xbipho.sender@gmail.com", password)
    msg = EmailMessage()

    html_table_blue_light = 

    df3 = df.copy()
    df3['link'] = df['link'].apply(lambda k: f'<a href="{k}">{k}</a>')
    df3['description'] = df['description'].apply(lambda k: k.replace('\n', '<br>'))
    df3['address'] = df['address'].apply(lambda k: Address.to_string(k).replace('\n', '<br>'))
    html = HTML_with_style(df3, '<style>table {{{}}}</style>'.format(my_style))

    message = f'''\n\
      Hi Maria and Leo,

      I have found the following appartements:

      {build_table(df3, 'blue_light')}
      '''
    msg.set_content(message)
    msg['Subject'] = f'I found {len(links)} new Appartements!'
    msg['From'] = 'xbipho.sender@gmail.com'
    msg['To'] = 'pohly3@gmail.com'
    
    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    server.send_message(msg)
message

In [ ]:
message

In [ ]:
my_style = """background-color: rgba(0, 0, 0, 0);
border-bottom-color: rgb(0, 0, 0);
border-bottom-style: none;
border-bottom-width: 0px;
border-collapse: collapse;
border-image-outset: 0px;
border-image-repeat: stretch;
border-image-slice: 100%;
border-image-source: none;
border-image-width: 1;
border-left-color: rgb(0, 0, 0);
border-left-style: none;
border-left-width: 0px;
border-right-color: rgb(0, 0, 0);
border-right-style: none;
border-right-width: 0px;
border-top-color: rgb(0, 0, 0);
border-top-style: none;
border-top-width: 0px;
box-sizing: border-box;
color: rgb(0, 0, 0);
display: table;
font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
font-size: 12px;
height: 1675px;
line-height: 20px;
margin-left: 0px;
margin-right: 0px;
margin-top: 12px;
table-layout: fixed;
text-size-adjust: 100%;
width: 700px;
-webkit-border-horizontal-spacing: 0px;
-webkit-border-vertical-spacing: 0px;
-webkit-tap-highlight-color: rgba(0, 0, 0, 0);"""


In [ ]:
def HTML_with_style(df, style=None, random_id=None):
    from IPython.display import HTML
    import numpy as np
    import re

    df_html = df.to_html()

    if random_id is None:
        random_id = 'id%d' % np.random.choice(np.arange(1000000))

    if style is None:
        style = """
        <style>
            table#{random_id} {{color: blue}}
        </style>
        """.format(random_id=random_id)
    else:
        new_style = []
        s = re.sub(r'</?style>', '', style).strip()
        for line in s.split('\n'):
                line = line.strip()
                if not re.match(r'^table', line):
                    line = re.sub(r'^', 'table ', line)
                new_style.append(line)
        new_style = ['<style>'] + new_style + ['</style>']

        style = re.sub(r'table(#\S+)?', 'table#%s' % random_id, '\n'.join(new_style))

    df_html = re.sub(r'<table', r'<table id=%s ' % random_id, df_html)

    return HTML(f"<html><head>{style}</head><body>{df_html}</body></html>").data

In [ ]:
from pretty_html_table import build_table

In [ ]:
html_table_blue_light

In [ ]:
# df.to_html('html.html', 
#            formatters = {'link': lambda k: f'<a href="{k}">{k}</a>',
#                          'description': lambda k: k.replace('\n', '<br>')}, 
#            escape=False,
#            index=False)
df3 = df.copy()
df3['link'] = df['link'].apply(lambda k: f'<a href="{k}">{k}</a>')
df3['description'] = df['description'].apply(lambda k: k.replace('\n', '<br>'))
df3['address'] = df['address'].apply(lambda k: Address.to_string(k).replace('\n', '<br>'))
html = HTML_with_style(df3, '<style>table {{{}}}</style>'.format(my_style))

In [ ]:
html